## Neural Network Attempt

In [1]:
from tables import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.regularizers import l2
import pandas as pd
import numpy as np

c:\python\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
train_labeled = pd.read_hdf("train_labeled.h5", "train")
train_unlabeled = pd.read_hdf("train_unlabeled.h5", "train")
test = pd.read_hdf("test.h5", "test")

#train_unlabeled.shape
train_labeled.head(5)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128
0,1,0.437475,0.617803,0.781499,0.404499,0.718066,0.554732,0.459540,0.579324,0.648441,...,0.523730,0.629020,0.494569,0.563879,0.604331,0.497247,0.335207,0.537755,0.555317,0.458929
1,0,0.436973,0.611919,0.787908,0.403480,0.725049,0.559196,0.460639,0.588320,0.657027,...,0.527735,0.636653,0.486114,0.555791,0.612709,0.491065,0.328115,0.536971,0.551187,0.473370
2,1,0.437700,0.619443,0.783668,0.405216,0.719925,0.556307,0.461546,0.582350,0.650066,...,0.519743,0.628397,0.498987,0.560861,0.607409,0.496805,0.334710,0.533678,0.555739,0.455152
3,7,0.436640,0.621330,0.783778,0.407233,0.721089,0.554641,0.462114,0.590494,0.658855,...,0.518988,0.639706,0.489974,0.562801,0.610364,0.496686,0.326092,0.537641,0.563333,0.465957
4,8,0.438236,0.607418,0.781259,0.411375,0.721206,0.550983,0.457552,0.585201,0.653511,...,0.528876,0.636095,0.487913,0.566749,0.599150,0.495933,0.325875,0.528958,0.560090,0.466333


In [3]:
y_train = train_labeled['y']
x_train = np.array(train_labeled._drop_axis(['y'], axis=1))
#x_train_unlabeled = np.array(train_unlabeled)
x_test = np.array(test)

#Convert to 1-hot for the model
#y_train_hot = np.zeros((y_train.shape[0], 10))
#y_train_hot[np.arange(y_train.shape[0]), y_train] = 1

### Apply mean normalization, much faster convergence

In [4]:
num_features = len(x_train[1])
total = 0

#standardize data, so for each x1..x100, do (x-mean)/sd(x) s.t. mean(x) ~= 0
for i in range(num_features):
    x_train[:,i] = (x_train[:,i] - np.mean(x_train[:,i])) / np.std(x_train[:,i])
    x_test[:,i] = (x_test[:,i] - np.mean(x_test[:,i])) / np.std(x_test[:,i])
    total += np.mean(x_train[:,i])


#adding all the means, we still have a very small # close to 0
total

-0.00013782128309358654

In [5]:
#Define model
model = Sequential([Dense(70, input_shape=(128,)), #, W_regularizer=l2(.01)
                   Activation('relu'),
                   Dense(50),
                   Activation('relu'),
                   Dense(30),
                   Activation('relu'),
                   Dense(10),
                   Activation('softmax'),
])

In [6]:
### Training
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

#Easy way to convert to one-hot
y_train_hot2 = keras.utils.to_categorical(y_train, num_classes=10)

model.fit(x_train, y_train_hot2, epochs=75)

Epoch 1/70
9000/9000 [==============================] - 1s 62us/step - loss: 0.9636 - acc: 0.6893
Epoch 2/70
9000/9000 [==============================] - 1s 71us/step - loss: 0.4250 - acc: 0.8648
Epoch 3/70
9000/9000 [==============================] - 1s 67us/step - loss: 0.3327 - acc: 0.8948
Epoch 4/70
9000/9000 [==============================] - 1s 68us/step - loss: 0.2742 - acc: 0.9100
Epoch 5/70
9000/9000 [==============================] - 1s 70us/step - loss: 0.2407 - acc: 0.9214
Epoch 6/70
9000/9000 [==============================] - 1s 71us/step - loss: 0.2113 - acc: 0.9316
Epoch 7/70
9000/9000 [==============================] - 1s 76us/step - loss: 0.1832 - acc: 0.9404
Epoch 8/70
9000/9000 [==============================] - 1s 76us/step - loss: 0.1684 - acc: 0.9461
Epoch 9/70
9000/9000 [==============================] - 1s 73us/step - loss: 0.1464 - acc: 0.9507
Epoch 10/70
9000/9000 [==============================] - 1s 76us/step - loss: 0.1330 - acc: 0.9567
Epoch 11/70
9000/90

In [8]:
pred_one_hot = model.predict(x_test)

#Convert predictions back from 1-hot
pred = []

for line in pred_one_hot:
    pred.append(np.argmax(line))
    
d = {'Id': test.index, 'y': pred}
out = pd.DataFrame(d)
out.to_csv('output.csv', index=False)